This is a notebook to serve as a repository for my Coursera IBM Data Science Capstone project.

In [3]:
import pandas as pd
import numpy as np

print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Begin Week 3 Toronto Clustering Project

Install libraries

In [4]:
!conda install -c anaconda scikit-learn --yes # import scikit-learn
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c anaconda BeautifulSoup4 --yes

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libgcc-ng:    7.2.0-h7cc24e2_2      --> 8.2.0-hdf63c60_1      anaconda
    libstdcxx-ng: 7.2.0-h7a57d05_2      --> 8.2.0-hdf63c60_1      anaconda
    mkl:          2018.0.0-hb491cac_4   --> 2018.0.3-1            anaconda
    scikit-learn: 0.19.1-py35hbf1f462_0 --> 0.20.0-py35h4989274_1 anaconda

libgcc-ng-8.2. 100% |################################| Time: 0:00:00  81.39 MB/s
libstdcxx-ng-8 100% |################################| Time: 0:00:00  66.23 MB/s
mkl-2018.0.3-1 100% |################################| Time: 0:00:03  66.41 MB/s
scikit-learn-0 100% |################################| Time: 0:00:00  68.96 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will 

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from lxml import html

print('Modules imported.')

Modules imported.


Begin process of scraping Wikipedia for Toronto neighborhoods

In [6]:
# Get requests function to scrape Wikipedia
import requests
import lxml.html as LH
from bs4 import BeautifulSoup

In [7]:
# Fetch the Wiki data
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

raw_wiki_page = requests.get(wiki_link) #get raw wiki page on Toronto neighborhoods

sourceCode = raw_wiki_page.content #get string of source code from response

wiki_page_html= html.document_fromstring(sourceCode) #make HTML element object

Use BeautifulSoup to parse the raw HTML into a useable Pandas dataframe

In [8]:
soup = BeautifulSoup(sourceCode, 'html.parser')

In [9]:
# Parse the HTML data into a list
neighborhood_names = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    neighborhood_names.append([ele for ele in cols if ele]) # Get rid of empty values

In [10]:
# Remove the first record to allow the transformation with the right dimensions
neighborhood_names.remove([])
print(neighborhood_names)

[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Islington'], ['M9B', 'Etobicoke', 'Martin Gro

In [11]:
np.shape(neighborhood_names)  # Check the shape of the list

(289, 3)

In [12]:
# Import the raw list into a Pandas dataframe
data = np.array(neighborhood_names)

df_neighborhoods = pd.DataFrame({'Postcode':data[:,0],'Borough':data[:,1], 'Neighborhood':data[:,2]})

df_neighborhoods.head() 


,Borough,Neighborhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [13]:
# Filter out non-assigned boroughs, PO boxes and funky PO drop zones

df_neighborhoods_filtered = df_neighborhoods[~df_neighborhoods['Borough'].isin(['Not assigned'])]

df_neighborhoods_filtered = df_neighborhoods_filtered[~df_neighborhoods_filtered['Neighborhood'].isin(['Stn A PO Boxes 25 The Esplanade', 'Business reply mail Processing Centre969 Eastern', 'Canada Post Gateway Processing Centre'])]

for row in range(len(df_neighborhoods_filtered)):
    postcode = df_neighborhoods_filtered.iloc[row,0]
    borough = df_neighborhoods_filtered.iloc[row,1] 
    neighborhood = df_neighborhoods_filtered.iloc[row,2]
    if neighborhood == "Not assigned":
        df_neighborhoods_filtered.iloc[row,2] = borough

df_neighborhoods_filtered = df_neighborhoods_filtered.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

df_neighborhoods_filtered = df_neighborhoods_filtered.sort_values(by=['Borough','Neighborhood'])

df_neighborhoods_filtered = df_neighborhoods_filtered.reset_index(drop=True)

print(df_neighborhoods_filtered.shape)
df_neighborhoods_filtered

(100, 3)


,Postcode,Borough,Neighborhood
0,M4S,Central Toronto,Davisville
1,M4P,Central Toronto,Davisville North
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
4,M4N,Central Toronto,Lawrence Park
5,M4T,Central Toronto,"Moore Park, Summerhill East"
6,M4R,Central Toronto,North Toronto West
7,M5N,Central Toronto,Roselawn
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
9,M5H,Downtown Toronto,"Adelaide, King, Richmond"


The geocoder library would not work at all, so I uploaded the latitude/longitude CSV file here.

In [14]:
# The code was removed by Watson Studio for sharing.

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now joining the geo-location data to the main neighborhood list.

In [15]:
df_toronto_neighborhoods = df_neighborhoods_filtered.join(df_latlong.set_index('Postcode'), on='Postcode')

df_toronto_neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4S,Central Toronto,Davisville,43.704324,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
df_toronto_neighborhoods.shape  # Checking the shape of the new file, still 100 rows

(100, 5)

In [17]:
df_toronto_neighborhoods.dtypes  # Checking the data types too, just in case.

Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

Now we create a map of Toronto with the cleaned up neighborhood list to make sure it works right.

In [18]:
# create map of Toronto using latitude and longitude values

# Toronto location:
latitude = 43.653963
longitude = -79.387207

neighborhoods = df_toronto_neighborhoods
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we get the Foursquare venue data and map it to the neighborhood list.

In [19]:
# Get Foursquare credentials

CLIENT_ID = 'DTLQASMVMMS0MQIAPDBGXP2JK3NVAOLOBFNJYWXYYZVEISHH' # your Foursquare ID
CLIENT_SECRET = 'FZ14MFTMLS405PUNLR0OUEKKW3W4OSS20MGOYO2QG1MC4NN0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DTLQASMVMMS0MQIAPDBGXP2JK3NVAOLOBFNJYWXYYZVEISHH
CLIENT_SECRET:FZ14MFTMLS405PUNLR0OUEKKW3W4OSS20MGOYO2QG1MC4NN0


In [20]:
# Use FS Explore API to get venues for top 100 venues within 500 meters of each neighborhood

radius = 500
LIMIT = 100

# Create getNearbyVenues function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# Get the venues for each postal code.

df_toronto_venues = getNearbyVenues(names=df_toronto_neighborhoods['Postcode'],
                                   latitudes=df_toronto_neighborhoods['Latitude'],
                                   longitudes=df_toronto_neighborhoods['Longitude']
                                  )

M4S
M4P
M4V
M5P
M4N
M4T
M4R
M5N
M5R
M5H
M5E
M5V
M4X
M5G
M5T
M6G
M4Y
M5L
M5K
M5X
M5S
M5J
M5A
M4W
M5B
M5C
M4M
M4E
M4L
M4K
M4J
M4G
M4H
M4B
M4C
M9V
M8W
M9C
M9B
M8V
M8Y
M9A
M9R
M8Z
M9W
M8X
M9P
M3H
M2K
M5M
M3K
M3B
M3M
M3N
M3L
M9M
M2J
M3C
M6B
M2H
M9L
M6A
M6L
M2M
M3J
M3A
M2L
M4A
M2N
M2R
M2P
M7A
M1S
M1V
M1N
M1H
M1L
M1T
M1M
M1P
M1K
M1E
M1C
M1W
M1R
M1B
M1J
M1X
M1G
M6K
M6H
M6P
M6J
M6R
M6S
M6E
M6M
M6C
M6N
M9N


In [22]:
print(df_toronto_venues.shape) # Checking the dimensions of the new venues dataframe.
df_toronto_venues.head()

(2124, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4S,43.704324,-79.38879,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
1,M4S,43.704324,-79.38879,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
2,M4S,43.704324,-79.38879,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
3,M4S,43.704324,-79.38879,XO Gelato,43.705177,-79.388793,Dessert Shop
4,M4S,43.704324,-79.38879,Viva Napoli,43.705752,-79.389125,Pizza Place


Next, we group the venues by postal code and count up the number of venues near each postal code location.

In [23]:
df_toronto_venues.groupby('Postcode').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,7,7,7,7,7,7
M1G,4,4,4,4,4,4
M1H,7,7,7,7,7,7
M1J,2,2,2,2,2,2
M1K,6,6,6,6,6,6
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


In [24]:
print('There are {} uniques categories.'.format(len(df_toronto_venues['Venue Category'].unique())))  # Looking to see how many venue categories we have.

There are 280 uniques categories.


Next, we use one-hot encoding to pivot the venue list and show the number of each type of venue near each postal code.

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = df_toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's St

In [26]:
toronto_onehot.shape # Checking the shape to make sure we still have at 270 venue types listed.

(2124, 281)

Then we covert the raw numbers to mean values to normalize them.

In [27]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head(10)

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's St

In [28]:
toronto_grouped.shape  # Checking shape again for number of categories, 270.

(97, 281)

Now, we use a couple of For loops to count up and rank the most common venue types for each postal code.

In [29]:
# Create a function to find the most common venue types
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
1,M1C,History Museum,Bar,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Dog Run
2,M1E,Spa,Medical Center,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Electronics Store,Dumpling Restaurant,Dog Run,Doner Restaurant
3,M1G,Coffee Shop,Insurance Office,Korean Restaurant,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Hakka Restaurant,Caribbean Restaurant,Bank,Donut Shop,Drugstore,Dumpling Restaurant
5,M1J,Spa,Playground,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
6,M1K,Discount Store,Train Station,Department Store,Coffee Shop,Convenience Store,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,M1L,Bakery,Bus Line,Fast Food Restaurant,Soccer Field,Park,Intersection,Metro Station,Bus Station,Eastern European Restaurant,Drugstore
8,M1M,Motel,American Restaurant,Yoga Studio,Discount Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,M1N,College Stadium,General Entertainment,Café,Skating Rink,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


Now we use the K-Means clustering technique to divide up the postal codes into 5 groups based on the number of venue types in each one.

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 3, 0, 4, 2,
       4, 0, 0, 0, 0, 4, 0, 0, 0, 2, 0, 4, 0, 0, 0, 4, 0], dtype=int32)

In [32]:
# Create a final merged dataframe that includes neighborhood name strings and geo-location data for mapping.
toronto_merged = neighborhoods_venues_sorted.merge(df_toronto_neighborhoods,on='Postcode',
                   how='inner')

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

toronto_merged = toronto_merged.drop('Borough',axis=1)

print(toronto_merged.shape)

toronto_merged.head() # check the last columns!

(97, 15)


,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood,Latitude,Longitude,Cluster Labels
0,M1B,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,"Rouge, Malvern",43.806686,-79.194353,0
1,M1C,History Museum,Bar,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Dog Run,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0
2,M1E,Spa,Medical Center,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Electronics Store,Dumpling Restaurant,Dog Run,Doner Restaurant,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0
3,M1G,Coffee Shop,Insurance Office,Korean Restaurant,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Woburn,43.770992,-79.216917,0
4,M1H,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Hakka Restaurant,Caribbean Restaurant,Bank,Donut Shop,Drugstore,Dumpling Restaurant,Cedarbrae,43.773136,-79.239476,0


Finally, we map out the color coded postal zones on the Toronto map to indicate where the most similar zones are located. It appears that, based on the large number of red zone markers for Cluster 0, that Toronto is a very homogeneous town as far as entertainment venue distribution. There is not a lot of difference between many of the neighborhoods according to this metric.

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters